# Minimal RNN notebook

### RNN representation

Contrary to a Feed Forward Neural Network, an RNN is a recurrent neural network, in which the information flow is not linear. A general representation can be seen as follows:

![Representation](img/rnn_simple.svg)

An RNN is useful to deal with sequential information: a sequence of inputs is fed through the network and the hidden state is updated at each step of the sequence. The sequence is commonly represented as a time sequence, and the most straight forward learning algorithm is backpropagation through time (BPTT) http://en.wikipedia.org/wiki/Backpropagation_through_time.

To understand properly BPTT, a better representation of the RNN is its unfolded version:

![Representation](img/rnn_unfolded.svg)

The input X is a sequence $x_0, x_1, ... x_t$, at each time-step t a new input $x_t$ is fed to the network.

### Equations

The most simple forward equations for a RNN are as follows:

$$h_t = \tanh(x_t . W_{in} + h_{t-1} . W_{rec})$$
$$y_t = softmax(h_t . W_{out})$$

Depending on the problem, all the outputs $y_0, ... y_t$ might be useful, or just $y_t$ the last one.

In [2]:
import numpy as np
import theano
import theano.tensor as T
from theano import shared 
from collections import OrderedDict

dtype=T.config.floatX
theano.config.optimizer='fast_compile'

In [3]:
def init_weight(shape, name, sample='uni'):
    if sample=='unishape':
        return shared(value=np.asarray(np.random.uniform(
                low=-np.sqrt(6. / (shape[0] + shape[1])),
                high=np.sqrt(6. / (shape[0] + shape[1])),
                size=shape), dtype=dtype), 
                    name=name, borrow=True)
    
    if sample=='svd':
        values = np.ndarray(shape, dtype=dtype)
        for dx in xrange(shape[0]):
            vals = np.random.uniform(low=-1., high=1.,  size=(shape[1],))
            values[dx,:] = vals
        _,svs,_ = np.linalg.svd(values)
        #svs[0] is the largest singular value                      
        values = values / svs[0]
        return shared(values, name=name, borrow=True)
    
    if sample=='uni':
        return shared(value=np.asarray(np.random.uniform(low=-0.1,high=0.1, size=shape), dtype=dtype), 
                      name=name, borrow=True)
    
    if sample=='zero':
        return shared(value=np.zeros(shape=shape, dtype=dtype), 
                      name=name, borrow=True)
    
    
    raise "error bad sample technique"

In [4]:
class Rnn:
    def __init__(self, n_in, n_hid, n_out, lr):   
        self.n_in = n_in
        self.n_hid = n_hid
        self.n_out = n_out
        self.W_in = init_weight((self.n_in, self.n_hid),'W_in', 'svd')
        self.W_out = init_weight((self.n_hid, self.n_out),'W_out', 'svd')
        self.W_rec = init_weight((self.n_hid, self.n_hid),'W_rec', 'svd')
        self.b_out = init_weight((self.n_out), 'b_out','zero')
        self.params = [self.W_in,self.W_out,self.W_rec, self.b_out]
        
        def step(x_t, h_tm1):
            h_t = T.nnet.tanh(T.dot(x_t, self.W_in) + T.dot(h_tm1, self.W_rec))
            y_t = T.nnet.softmax(- (T.dot(h_t, self.W_out) + self.b_out))            
            return [h_t, y_t]

        X = T.matrix() # X is the sequence of vector
        Y = T.matrix() # Y is the output of vector
        h0 = shared(np.zeros(self.n_hid, dtype=dtype)) # initial hidden state 
        y0 = shared(np.ones(self.n_out, dtype=dtype)) # starting output sequence
        lr = shared(np.cast[dtype](lr))
        
        [h_vals, y_vals], _ = theano.scan(fn=step,                                  
                                          sequences=X,
                                          outputs_info=[h0, None])
                
        cost = -T.mean(Y * T.log(y_vals)+ (1.- Y) * T.log(1. - y_vals))
        cost = T.nnet.binary_crossentropy(y_vals, Y)
        
        gparams = T.grad(cost, self.params)
        updates = OrderedDict()
        for param, gparam in zip(self.params, gparams):
            updates[param] = param - gparam * lr
        
        self.train = theano.function(inputs = [X, Y], outputs = cost, updates=updates)
        self.predictions = theano.function(inputs = [X], outputs = y_vals)                
    

In [7]:
X = T.tensor3()
Y = T.tensor3()

bc = theano.function(inputs = [X,Y], outputs = T.nnet.binary_crossentropy(X,Y))

In [15]:
x = np.random.uniform(size=(10,5,8))
y = np.random.uniform(size=(10,5,8))
bc(x,y).shape

(10, 5, 8)

In [ ]:
class RnnMiniBatch:
    def __init__(self, n_in, n_hid, n_out, lr):   
        self.n_in = n_in
        self.n_hid = n_hid
        self.n_out = n_out
        self.W_in = init_weight((self.n_in, self.n_hid),'W_in')
        self.W_out = init_weight((self.n_hid, self.n_out),'W_out')
        self.W_rec = init_weight((self.n_hid, self.n_hid),'W_rec')
        
        
        self.params = [self.W_in,self.W_out,self.W_rec]
        
        def step(x_t, h_tm1):
            h_t = T.nnet.sigmoid(T.dot(x_t, self.W_in) + T.dot(h_tm1, self.W_rec))
            y_t = T.nnet.softmax(T.dot(h_t, self.W_out))
            return [h_t, y_t]


        X = T.tensor3() # batch of sequence of vector
        Y = T.matrix() # batch of output vector 
        h0 = shared(np.zeros(self.n_hid, dtype=dtype)) # initial hidden state 
        y0 = shared(np.ones(self.n_out, dtype=dtype)) # starting output sequence
        lr = shared(np.cast[dtype](lr))
        
        [h_vals, y_vals], _ = theano.scan(fn=step,                                  
                                          sequences=X,
                                          outputs_info=[h0, None])
        
        
        cost = - T.mean(Y * T.log(y_vals[-1,0,:]) + (1- Y) * T.log(1.-y_vals[-1,0,:]))
        gparams = T.grad(cost, self.params)
        updates = OrderedDict()
        for param, gparam in zip(self.params, gparams):
            updates[param] = param - gparam * lr
        
        self.train = theano.function(inputs = [X, Y], outputs = cost, updates=updates)
        self.predictions = theano.function(inputs = [X], outputs = y_vals[-1,0,:])
        
        '''y_vals[:,-1]
        y_pred = T.argmax(self.p_y_given_x, axis=1)
        cost = - T.mean()
        
        gibbs10 = theano.function([sample], values[-1], updates=updates)'''
    

In [5]:
model = Rnn(7, 50, 7, 0.1)

/usr/local/lib/python2.7/dist-packages/Theano-0.6.0-py2.7.egg/theano/tensor/opt.py:2213: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.
  if (replace_x == replace_y and
/usr/local/lib/python2.7/dist-packages/Theano-0.6.0-py2.7.egg/theano/scan_module/scan_perform_ext.py:117: RuntimeWarning: numpy.ndarray size changed, may indicate binary incompatibility
  from scan_perform.scan_perform import *


In [ ]:
X = np.random.uniform(low=-0.1, high=0.1, size=(15,7)).astype(dtype=dtype) 
model.predictions(X)


In [ ]:

X = np.random.uniform(low=-0.1, high=0.1, size=(100,15,10)).astype(dtype=dtype) 
Y = np.zeros(shape=(100,5)).astype(dtype=dtype)
indices = np.random.randint(5, size=(100))
for x in range(Y.shape[0]):
    Y[x,indices[x]]=1.

In [ ]:
model.train(X,Y)

In [ ]:
nb_epochs = 100
#stupid and naive sgd
for x in range(nb_epochs):
    error = 0.
    for j in range(len(train_data)):  
        index = np.random.randint(0, len(train_data))
        i, o = train_data[index]
        train_cost = model.train(i, o)
        error += train_cost
    if x%10==0:
            print "epoch "+str(x)+ " error: "+str(error)

In [ ]:
import numpy as np

chars='BTSXPVE'

graph = [[(1,5),('T','P')] , [(1,2),('S','X')], \
           [(3,5),('S','X')], [(6,),('E')], \
           [(3,2),('V','P')], [(4,5),('V','T')] ]


def in_grammar(word):
    if word[0] != 'B':
        return False
    node = 0    
    for c in word[1:]:
        transitions = graph[node]
        try:
            node = transitions[0][transitions[1].index(c)]
        except ValueError: # using exceptions for flow control in python is common
            return False
    return True        
      
def sequenceToWord(sequence):
    """
    converts a sequence (one-hot) in a reber string
    """
    reberString = ''
    for s in sequence:
        index = np.where(s==1.)[0][0]
        reberString += chars[index]
    return reberString
    
def generateSequences(minLength):
    while True:
        inchars = ['B']
        node = 0
        outchars = []    
        while node != 6:
            transitions = graph[node]
            i = np.random.randint(0, len(transitions[0]))
            inchars.append(transitions[1][i])
            outchars.append(transitions[1])
            node = transitions[0][i]
        if len(inchars) > minLength:  
            return inchars, outchars


def get_one_example(minLength):
    inchars, outchars = generateSequences(minLength)
    inseq = []
    outseq= []
    for i,o in zip(inchars, outchars): 
        inpt = np.zeros(7)
        inpt[chars.find(i)] = 1.     
        outpt = np.zeros(7)
        for oo in o:
            outpt[chars.find(oo)] = 1.
        inseq.append(inpt)
        outseq.append(outpt)
    return inseq, outseq


def get_char_one_hot(char):
    char_oh = np.zeros(7)
    for c in char:
        char_oh[chars.find(c)] = 1.
    return [char_oh] 
    
def get_n_examples(n, minLength=10):
    examples = []
    for i in xrange(n):
        examples.append(get_one_example(minLength))
    return examples

emb_chars = "TP"


def get_one_embedded_example(minLength=10):
    i, o = get_one_example(minLength)
    emb_char = emb_chars[np.random.randint(0, len(emb_chars))]
    new_in = get_char_one_hot(('B',))
    new_in += get_char_one_hot((emb_char,))
    new_out= get_char_one_hot(emb_chars)
    new_out+= get_char_one_hot('B',)
    new_in += i
    new_out += o
    new_in += get_char_one_hot(('E',))
    new_in += get_char_one_hot((emb_char,))
    new_out += get_char_one_hot((emb_char, ))
    new_out += get_char_one_hot(('E',))
    return new_in, new_out
    
def get_n_embedded_examples(n, minLength=10):
    examples = []
    for i in xrange(n):
        examples.append(get_one_embedded_example(minLength))
    return examples

In [ ]:
train_data = get_n_embedded_examples(1000)

In [ ]:
test_data = get_n_embedded_examples(10)

def print_out(test_data):
    for i,o in test_data:
        p = model.predictions(i)
        print o[-2] # target
        print np.asarray([0. if x!=np.argmax(p[-2]) else 1. for x in range(7)]) # prediction
        print 
print_out(test_data)

In [ ]:
[len(x[0]) for x in test_data]

In [ ]:
# word prediction
import re
import random
import numpy as np
from gensim import corpora


def process(x):
    return re.sub('\W+', ' ', x).lower().split()


class Corpus:
    def __init__(self, seq_x=None, dic=None):                
        self.seq_x = []
        self.seq_y = []
        self.matrix = []
        self.idx2word = {}
        self.word2idx = {}
        if dic == None:
            dictionary = corpora.Dictionary(process(line) for line in TextList + TitleList)
            dictionary.filter_extremes(no_below=10,no_above=1.0, keep_n=100000)
            dictionary.compactify()
            self.idx2word = {k:v for (k,v) in dictionary.items()}
            self.idx2word[len(self.idx2word)] = 'END'
            self.word2idx = {v:k for (k,v) in self.idx2word.items()}
            del dictionary
        else:
            self.idx2word = dic
            self.idx2word[len(self.idx2word)] = 'END'
            self.word2idx = {v:k for (k,v) in self.idx2word.items()}
        self.vocsize = len(self.idx2word)

        if seq_x!=None:
            for line in seq_x:
                words = filter(lambda w: w in self.word2idx, process(line))
                self.seq_x.append(words)
        '''for line in seq_y:
            words = line.split()
            self.seq_y.append(words)
            words = filter(lambda w: w in dictionary, process(line))
            self.seq_x.append(words)  
            for word in words:
                dic_freq[word] = dic_freq.get(word, 0) + 1'''
        
    def to_numpy(self):
        
        correct_seqs = [seq for seq in self.seq_x if len(seq) > 99]
        self.matrix = np.zeros(shape=(len(correct_seqs), 100), dtype='int32')
        for idx, seq in enumerate(correct_seqs):
            seq_idxs = [self.word2idx[w] for w in seq[:100]]
            if len(seq_idxs)<100:
                continue
            row = np.asarray(seq_idxs, dtype='int32')
            self.matrix[idx,:] = row
        return self.matrix
        
    def one_hot(self, x):
        vec = np.zeros(size=(1,1,self.vocsize), dtype=dtype)
        vec[1,1,x] = 1.0
        return vec

def make_dataset(matrix, pad, start=3, min_len=10, max_len=20):      
    assert(start+max_len<matrix.shape[1])
    dataset_x = np.ones(shape = (matrix.shape[0], max_len), dtype = 'int32') * pad
    dataset_y = np.zeros(shape = (matrix.shape[0]), dtype = 'int32')        
    for idx in range(matrix.shape[0]):
        length = random.randint(min_len, max_len)

        #pad with end seq                        
        dataset_x[idx,0:length] = matrix[idx,start:start+length]    
        dataset_y[idx] = matrix[idx,length]
    return [dataset_x, dataset_y]
        #voc = [k for (k,v) in dic_freq.items() if v>=min_freq]
        #print "loaded "+ len(dic_freq) + "words, kept " + len(voc) + "words"
        #self.idx_voc = {v:k for (k,v) in self.voc_idx.items()}
        
    #todo save / load
            

In [ ]:
import json
filename = "/media/charles/data/articles"
h = open(filename)
all_jsons=[]
for line in h:    
    if line[0]=='[':
        all_jsons.append(line[:-1])
        
TitleList = []
TextList = []
IndexList = []
count = 0

for oneJson in all_jsons:
    u = json.loads(oneJson)
    for item in u:
        fields = item['fields']
        TitleList.append(fields['title'])
        TextList.append(fields['text'])
        IndexList.append(item['rowKey'])
        count+=1
        if count%10000==0:
            print("done: "+str(count))

all_jsons = []
del all_jsons


In [ ]:
len(TitleList), len(TextList)
seq_x = corpus.seq_x
dictionary = corpora.Dictionary(process(line) for line in TextList + TitleList)
dictionary.filter_extremes(no_below=10,no_above=1.0, keep_n=100000)
dictionary.compactify()
dic = {k:v for (k,v) in dictionary.items()}

In [ ]:
corpus = Corpus(seq_x=TextList, dic=None)
#[dx, dy] = corpus.make_dataset()

In [ ]:
matrix = corpus.to_numpy()


In [18]:
class RnnMiniBatch:
    def __init__(self, n_in, n_hid, n_out, lr):   
        self.n_in = n_in
        self.n_hid = n_hid
        self.n_out = n_out
        self.W_in = init_weight((self.n_in, self.n_hid),'W_in')
        self.W_out = init_weight((self.n_hid, self.n_out),'W_out')
        self.W_rec = init_weight((self.n_hid, self.n_hid),'W_rec', 'svd')
                
        self.params = [self.W_in,self.W_out,self.W_rec]
        
        def step(x_t, h_tm1):
            h_t = T.nnet.sigmoid(T.dot(x_t, self.W_in) + T.dot(h_tm1, self.W_rec))
            y_t = T.nnet.softmax(T.dot(h_t, self.W_out))
            return [h_t, y_t]


        X = T.tensor3() # batch of sequence of vector
        Y = T.tensor3() # batch of sequence of vector (should be 0 when X is not null) 
        print X.shape
        h0 = shared(np.zeros(shape=(X.shape[0],self.n_hid), dtype=dtype)) # initial hidden state         
        mask = 1. - X.sum(axis = 1)
        lr = shared(np.cast[dtype](lr))
        
        [h_vals, y_vals], _ = theano.scan(fn=step,        
                                          sequences=X,
                                          outputs_info=[h0, None])
        
        cost = (T.nnet.binary_crossentropy(y_vals, Y) * mask).sum()        
        gparams = T.grad(cost, self.params)
        updates = OrderedDict()
        for param, gparam in zip(self.params, gparams):
            updates[param] = param - gparam * lr
        
        self.train = theano.function(inputs = [X, Y], outputs = cost, updates=updates)
        self.predictions = theano.function(inputs = [X], outputs = y_vals[-1,0,:])
    


In [19]:
z = RnnMiniBatch(3, 4, 3, 0.4)

Shape.0


TypeError: an integer is required